### Clinical BCI Challenge-WCCI2020
- [website link](https://sites.google.com/view/bci-comp-wcci/?fbclid=IwAR37WLQ_xNd5qsZvktZCT8XJerHhmVb_bU5HDu69CnO85DE3iF0fs57vQ6M)


 - [Dataset Link](https://github.com/5anirban9/Clinical-Brain-Computer-Interfaces-Challenge-WCCI-2020-Glasgow)
 

In [1]:
import mne
import pywt
import scipy
import sklearn
import numpy as np
import pandas as pd
import os
import glob
from mne.decoding import CSP
from scipy.io import loadmat

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedShuffleSplit 
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.pipeline import make_pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.preprocessing import StandardScaler

In [4]:
import warnings
warnings.filterwarnings('ignore') # to ignore warnings

In [5]:
verbose = False                    # global variable to suppress output display of MNE functions
mne.set_log_level(verbose=verbose) # to suppress large info outputs

In [6]:
# using kappa as evaluation metric
kappa = sklearn.metrics.make_scorer(sklearn.metrics.cohen_kappa_score) # kappa scorer
acc = sklearn.metrics.make_scorer(sklearn.metrics.accuracy_score)      # accuracy scorer
scorer = kappa          # just assign another scorer to replace kappa scorer

In [7]:
n_jobs = -1  # for multicore parallel processing, set it to 1 if cause memory issues

## Data Loading and Conversion to MNE Datatypes
[Mike Cohen Tutorials link for EEG Preprocessing](https://www.youtube.com/watch?v=uWB5tjhataY&list=PLn0OLiymPak2gDD-VDA90w9_iGDgOOb2o)

In [8]:
current_folder = globals()['_dh'][0]  # a hack to get path of current folder in which juptyter file is located
data_path = os.path.join(current_folder, 'Data')

In [ ]:
training_files   = glob.glob(data_path + '/*T.mat')
len(training_files)     # if  return zero,then no file is loaded

In [10]:
def get_mne_epochs(filepath, verbose=verbose, t_start=2, fs=512, mode='train'):
    '''
    This function reads the EEG data from .mat file and convert it to MNE-Python Compatible epochs
    data structure. It takes data from [0, 8] sec range and return it by setting t = 0 at cue onset
    i.e. 3 seconds and dropping first two seconds so the output data is in [-1.0, 5.0] sec range. The
    Details can be found in the preprocessing section of the attached document
    '''
    mat_data = loadmat(filepath) # read .mat file
    eeg_data= mat_data['RawEEGData']
    idx_start = fs*t_start      
    eeg_data = eeg_data[:, :, idx_start:]
    event_id = {'left-hand': 1, 'right-hand': 2}
    channel_names = ['F3', 'FC3', 'C3', 'CP3', 'P3', 'FCz', 'CPz', 'F4', 'FC4', 'C4', 'CP4', 'P4']
    info = mne.create_info(ch_names=channel_names, sfreq=fs, ch_types='eeg')
    epochs = mne.EpochsArray(eeg_data, info, verbose=verbose, tmin=t_start-3.0)
    epochs.set_montage('standard_1020')
    epochs.filter(1., None) 
    epochs.apply_baseline(baseline=(-.250, 0)) # linear baseline correction
    
    if mode == 'train': # this in only applicable for training data
        epochs.event_id = event_id
        epochs.events[:,2] = mat_data['Labels'].ravel()    
    return epochs 

def get_labels(filepath):
    mat_data = loadmat(filepath) # read .mat file
    return mat_data['Labels'].ravel()

In [11]:
epochs, labels = get_mne_epochs(training_files[0], verbose=verbose), get_labels(training_files[0])
data = epochs.get_data()
print('Shape of EEG Data: ', data.shape, '\t Shape of Labels: ', labels.shape) 

Shape of EEG Data:  (80, 12, 3072) 	 Shape of Labels:  (80,)


## Lets Append Epochs

In [13]:
def get_mne_epochs_complete(files_paths, verbose=verbose, t_start=2, fs=512, mode='train'):
    '''
    similar to get_mne_epochs, just appends data from all relevant files together to give a single
    epoch object
    '''
    eeg_data = []
    for filepath in files_paths:
        mat_data = loadmat(filepath)
        eeg_data.extend(mat_data['RawEEGData'])

    idx_start = fs*t_start      # fs*ts
    eeg_data = np.array(eeg_data)
    eeg_data = eeg_data[:, :, idx_start:]
    event_id = {'left-hand': 1, 'right-hand': 2}
    channel_names = ['F3', 'FC3', 'C3', 'CP3', 'P3', 'FCz', 'CPz', 'F4', 'FC4', 'C4', 'CP4', 'P4']
    info = mne.create_info(ch_names=channel_names, sfreq=fs, ch_types='eeg')
    epochs = mne.EpochsArray(eeg_data, info, verbose=verbose, tmin=t_start-3.0)
    epochs.set_montage('standard_1020')
    epochs.filter(1., None) # required be ICA, (7-30 Hz) later
    epochs.apply_baseline(baseline=(-.250, 0)) # linear baseline correction
    
    if mode == 'train': # this in only applicable for training data
        labels = []
        for filepath in files_paths:
            mat_data = loadmat(filepath)
            labels.extend(mat_data['Labels'].ravel())
        epochs.event_id = event_id
        epochs.events[:,2] = labels    
    return epochs 

### Data Loading with Band Pass Filtering

In [14]:
# loading relevant files
training_epochs_all = get_mne_epochs_complete(training_files).filter(7,32)

## Leave One Group Out CV

In [15]:
# group parameter for leave one group out cross validation in sklearn, each subject is given unique identifier
group_list = []
for subject in np.linspace(1,8,8):
    group_list.extend([subject for _ in range(80)]) # since we have 80 samples in each training file
groups = np.array(group_list)

In [16]:
cv = LeaveOneGroupOut()

## Wavelet Packet Decomposition WPD

In [16]:
def wpd(X, wavelet ='coif1', maxlevel=3): 
    coeffs = pywt.WaveletPacket(X, wavelet ,mode='symmetric',maxlevel=maxlevel) # coif1 is recommended by a paper
    return coeffs

In [17]:
def get_wpd_coeffs(x, wavelet='coif1', wpd_levels=3, order='natural'):
    '''
    expects data in (trials, channels, raw_data) and return in (trials, bands, channels, wpd_coeffs) for sklearn compatibility
    '''
    
    num_bands = 2**wpd_levels
    trials, channels = x.shape[0], x.shape[1]
    
    # first find the last dim size, bcz it is different for different levels and different mother wavelets
    coeff_size = wpd(x[0,0,:], wavelet, wpd_levels).get_level(wpd_levels)[0].data.shape[0]
    
    # we shall fill this array (trials, bands, channels, eegdata)
    bands = np.empty((trials, num_bands, channels, coeff_size))
       
    for trial in range(trials):
        for ch in range(channels):
            pos = []
            coeff = wpd(x[trial,ch,:], wavelet, wpd_levels) 
            nodes_paths = [node.path for node in coeff.get_level(level=wpd_levels, order=order)]
            
            for band, path in enumerate(nodes_paths): 
                bands[trial,band,ch,:] = coeff[path].data 
        
    return bands

In [18]:
def calculate_avg_power_wpd(data):
    trials, bands, channels, N = data.shape
    power = np.empty(shape=(trials, bands, channels))
    
    for trial in range(trials):
        for band in range(bands):
            for ch in range(channels):
                power[trial,band,ch] = (sum(data[trial,band,ch,:]**2))/N
    return power      

In [19]:
def calculate_statistics_wpd(data, ax=-1):
    mean = np.nanmean(data, axis=ax)
    std = np.nanstd(data, axis=ax)
    return np.stack([mean, std], axis=ax)

In [20]:
def get_wpd_features(data, wavelet='coif2', level=5, retained_coeffs = 'All', stat_feat=True, power_feat=True):
    '''
    takes (trials, channels, data) as input and returns (trials, bands, channels, features) as output
    has flexibility to control which set of features to use
    '''
    wpd_coeffs = get_wpd_coeffs(data,  wavelet=wavelet, wpd_levels=level)
    
    if retained_coeffs != 'All':
        wpd_coeffs = wpd_coeffs[:,retained_coeffs,:,:]
        
    if power_feat == True and stat_feat == True:        # both power and statistical features
        power = np.expand_dims(calculate_avg_power_wpd(wpd_coeffs), axis=-1)
        statistics = calculate_statistics_wpd(wpd_coeffs)
        band_features = np.concatenate((power, statistics), axis=-1)
    elif power_feat == True and stat_feat == False:     # only power features
        power = np.expand_dims(calculate_avg_power_wpd(wpd_coeffs), axis=-1)
        band_features = power
    elif power_feat == False and stat_feat == True:     # only statistical features
        statistics = calculate_statistics_wpd(wpd_coeffs)
        band_features = statistics
    elif power_feat == False and stat_feat == False:    # invalid case
        print('Cannot Set Both Power and Statistical Features to False')
        
    return band_features

## Discrete Wavelet Transform DWT
wavedec returns [cA_n, cD_n, cD_n-1, ..., cD2, cD1]

In [21]:
def dwt(data, wavelet='coif1', level=3):
    coeffs = pywt.wavedec(data, wavelet=wavelet, level=level)
    return coeffs

In [22]:
def calculate_avg_power_dwt(data):
    # expects individual coefficient of DWT as input
    trials, channels, N = data.shape
    power = np.empty(shape=(trials, channels))
    
    for trial in range(trials):
        for ch in range(channels):
            power[trial,ch] = (sum(data[trial,ch,:]**2))/N # division by N converts energy to power
    return power

In [23]:
# if you only want to use power features then use this function 
def get_dwt_power_features(data, wavelet='coif1', level=3, retained_coeffs = 'All'):
    '''
    takes (trials, channels, data) as input and returns (trials, bands, channels) as output
    '''
    dwt_coeffs = pywt.wavedec(data, wavelet=wavelet, level=level)
    if retained_coeffs == 'All':
        energy_dwt = np.stack(arrays = ([calculate_avg_power_dwt(dwt_coeffs[i]) for i in range(level+1)]), axis=1)
    else: 
        energy_dwt = np.stack(arrays = ([calculate_avg_power_dwt(dwt_coeffs[i]) for i in retained_coeffs]), axis=1)
    return energy_dwt

In [24]:
def calculate_statistics_dwt(data, ax=-1):
    mean = np.nanmean(data, axis=ax)
    std = np.nanstd(data, axis=ax)
    return np.stack([mean, std], axis=ax)

In [25]:
def get_dwt_features(data, wavelet='coif2', level=5, retained_coeffs = 'All', stat_feat=True, power_feat=True):
    '''
    takes (trials, channels, data) as input and returns (trials, bands, channels, features) as output
    has flexibility to control which set of features to use
    '''
    dwt_coeffs = pywt.wavedec(data, wavelet=wavelet, level=level)
    dummy_list = []
    
    if retained_coeffs == 'All':
        iter_list = range(len(dwt_coeffs))
    else:
        iter_list = retained_coeffs
    
    for i in iter_list:
        
        if power_feat == True and stat_feat == True:         # both power and statistical features
            power = np.expand_dims(calculate_avg_power_dwt(dwt_coeffs[i]), axis=-1)
            statistics = calculate_statistics_dwt(dwt_coeffs[i])
            band_features = np.concatenate((power, statistics), axis = -1)
        elif power_feat == True and stat_feat == False:     # only power features
            power = np.expand_dims(calculate_avg_power_dwt(dwt_coeffs[i]), axis=-1)
            band_features = power
        elif power_feat == False and stat_feat == True:     # only statistical features
            statistics = calculate_statistics_dwt(dwt_coeffs[i])
            band_features = statistics
        elif power_feat == False and stat_feat == False:    # invalid case
            print('Cannot Set Both Power and Statistical Features to False')
            break
        
        dummy_list.append(band_features)
        
    return np.stack(dummy_list, axis=1)

### Designing a custom transformer 
that can combine csp and reshape the wpd_data/dwt_data from (traila, bands, channels, data) to  (trials, channels, data) by applying CSP individually on each band and then concatenating the results

In [26]:
from sklearn.preprocessing import StandardScaler
from mne.decoding import CSP
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np
import scipy

class Custom_WT_CSP(TransformerMixin, BaseEstimator):
    """
    Expects data in the format (trials, bands, channels, wavelet_coeffs_data)
    individually apply CSP on each band and then concatenate to give output of the form (trials, channels, csp_filtered_data)
    """
    def __init__(self, csp_components=4):
        self.csp_components = csp_components       # csp components to retain
        self.num_bands = 0                         # captures the total frequency bands in filtered data
        self.Csp = []                              # would carry list of CSP's applied individually to each freq band
        
    def fit(self, x, y):
        self.num_bands = x.shape[1]
        self.Csp = [CSP(n_components=self.csp_components, reg=None, log=True, norm_trace=False) for _ in range(self.num_bands)]
        [self.Csp[i].fit(x[:,i,:,:], y) for i in range(self.num_bands)]
        return self
    
    def transform(self, x, y=None):
        return np.concatenate(tuple(self.Csp[i].transform(x[:,i,:,:]) for i in range(self.num_bands)),axis=-1)

### Helper Functions to Check which Wavelet to Use

In [27]:
wavelets = ['haar', 'db2', 'db4', 'coif1', 'coif2', 'coif4', 'sym2', 'sym4']
level_wpd = 3
level_dwt = 5

In [28]:
# a function to calculate mean scores across the classifiers
def find_mean_score(data, labels, scorer=kappa, csp_comps=4):
    classifiers = [
        LinearSVC(random_state=0),
        LDA(solver='eigen', shrinkage='auto')]

    scores = []
    for clf in classifiers:
        scores.append(np.mean(cross_val_score(make_pipeline(Custom_WT_CSP(csp_comps), StandardScaler(), clf), data, labels, scoring=scorer, cv=5)))
    return np.round(np.mean(scores), 3)

In [29]:
def get_best_wavelet(data, labels, wavelets, level_dwt=5, level_wpd=3, csp_comps=4):
    scores = []
    for wavelet in wavelets:
        dwt_features = get_dwt_features(data, wavelet=wavelet, level=level_dwt)
        wpd_features = get_wpd_features(data, wavelet=wavelet, level=level_wpd)
        combo_features = np.concatenate((wpd_features, dwt_features), axis=1)
        scores.append(find_mean_score(combo_features, labels, csp_comps=csp_comps))
    best_wavelet = wavelets[np.argmax(scores)]
    print('Best Wavelet:', best_wavelet, '\tBest Score:', max(scores))    
    return best_wavelet     

In [81]:
# for all training subjects
epochs = training_epochs_all
data, labels = epochs.get_data(), epochs.events[:,-1]
data = data[:,:,512+256:-256]            # to capture data from 0.5s to 4.5s range
best_wavelet = get_best_wavelet(data, labels, wavelets, csp_comps=4)

Best Wavelet: coif2 	Best Score: 0.288


In [83]:
best_wavelet = 'coif2'

## Performance with Default Settings

In [84]:
i = 4  # csp components

In [85]:
# from 0.5-4.5 sec
epochs = training_epochs_all.copy()
data, labels = epochs.get_data(), epochs.events[:,-1]
data = data[:,:,256+512:-256]
dwt_features = get_dwt_features(data, wavelet= best_wavelet, level=level_dwt)
wpd_features = get_wpd_features(data, wavelet= best_wavelet, level=level_wpd)
combo_wavelet_features = np.concatenate((wpd_features, dwt_features), axis=1)
x_train = combo_wavelet_features
y_train = labels
print('-'*10, 'Classification Scores Comparison with Default Settings', 10*'-')
print('KNN           : ', np.mean(cross_val_score(make_pipeline(Custom_WT_CSP(i), StandardScaler(), KNeighborsClassifier()), x_train, y_train, cv=cv, scoring=scorer, groups=groups)))
print('Log-Regression: ', np.mean(cross_val_score(make_pipeline(Custom_WT_CSP(i), StandardScaler(), LogisticRegression(max_iter=1000)), x_train, y_train, cv=cv, scoring=scorer, groups=groups)))
print('Linear SVM    : ', np.mean(cross_val_score(make_pipeline(Custom_WT_CSP(i), StandardScaler(), LinearSVC(random_state=0)), x_train, y_train, cv=cv, scoring=scorer, groups=groups)))
print('kernal SVM    : ', np.mean(cross_val_score(make_pipeline(Custom_WT_CSP(i), StandardScaler(), SVC(gamma='scale')), x_train, y_train, cv=cv, scoring=scorer, groups=groups)))
print('LDA           : ', np.mean(cross_val_score(make_pipeline(Custom_WT_CSP(i), StandardScaler(), LDA()), x_train, y_train, cv=cv, scoring=scorer, groups=groups)))

---------- Classification Scores Comparison with Default Settings ----------
KNN           :  0.184375
Log-Regression:  0.28125
Linear SVM    :  0.271875
kernal SVM    :  0.278125
LDA           :  0.2875


In [87]:
# from 1.5-4.5 sec, best one
epochs = training_epochs_all.copy()
data, labels = epochs.get_data(), epochs.events[:,-1]
data = data[:,:,256+512+512:-256]
dwt_features = get_dwt_features(data, wavelet= best_wavelet, level=level_dwt)
wpd_features = get_wpd_features(data, wavelet= best_wavelet, level=level_wpd)
combo_wavelet_features = np.concatenate((wpd_features, dwt_features), axis=1)
x_train = combo_wavelet_features
y_train = labels
print('-'*10, 'Classification Scores Comparison with Default Settings', 10*'-')
print('KNN           : ', np.mean(cross_val_score(make_pipeline(Custom_WT_CSP(i), StandardScaler(), KNeighborsClassifier()), x_train, y_train, cv=cv, scoring=scorer, groups=groups)))
print('Log-Regression: ', np.mean(cross_val_score(make_pipeline(Custom_WT_CSP(i), StandardScaler(), LogisticRegression(max_iter=1000)), x_train, y_train, cv=cv, scoring=scorer, groups=groups)))
print('Linear SVM    : ', np.mean(cross_val_score(make_pipeline(Custom_WT_CSP(i), StandardScaler(), LinearSVC(random_state=0)), x_train, y_train, cv=cv, scoring=scorer, groups=groups)))
print('kernal SVM    : ', np.mean(cross_val_score(make_pipeline(Custom_WT_CSP(i), StandardScaler(), SVC(gamma='scale')), x_train, y_train, cv=cv, scoring=scorer, groups=groups)))
print('LDA           : ', np.mean(cross_val_score(make_pipeline(Custom_WT_CSP(i), StandardScaler(), LDA()), x_train, y_train, cv=cv, scoring=scorer, groups=groups)))

---------- Classification Scores Comparison with Default Settings ----------
KNN           :  0.1875
Log-Regression:  0.296875
Linear SVM    :  0.28437500000000004
kernal SVM    :  0.31875
LDA           :  0.296875


## Grid Search
[1.5,4.5] sec window

In [113]:
epochs = training_epochs_all.copy()
data, labels = epochs.get_data(), epochs.events[:,-1]
data = data[:,:,256+512+512:-256] # from 1.5 sec to 4.5 sec
dwt_features = get_dwt_features(data, wavelet= best_wavelet, level=level_dwt)
wpd_features = get_wpd_features(data, wavelet= best_wavelet, level=level_wpd)
combo_wavelet_features = np.concatenate((wpd_features, dwt_features), axis=1)
x_train = combo_wavelet_features
y_train = labels

In [89]:
# for kNN
param_grid_knn = {'kneighborsclassifier__n_neighbors': np.arange(2,15,2),
                 'custom_wt_csp__csp_components': csp_comps}
grid_knn = GridSearchCV(make_pipeline(Custom_WT_CSP(), StandardScaler(), KNeighborsClassifier()), 
                        param_grid=param_grid_knn, cv=cv, scoring=scorer, n_jobs=n_jobs)
grid_knn.fit(x_train, y_train, groups=groups) 
print('Maximum Cross Validation Accuracy: {:.3f}'.format(grid_knn.best_score_))
print('Optimal Parameters: ', grid_knn.best_params_)

Maximum Cross Validation Accuracy: 0.325
Optimal Parameters:  {'custom_wt_csp__csp_components': 4, 'kneighborsclassifier__n_neighbors': 12}


In [90]:
# for logistic regression
param_grid_log_reg = {'logisticregression__C' : np.logspace(-4, 3, 15),
                     'custom_wt_csp__csp_components': csp_comps}
grid_log_reg = GridSearchCV(make_pipeline(Custom_WT_CSP(), StandardScaler(), LogisticRegression()), 
                            param_grid=param_grid_log_reg, cv=cv, scoring=scorer) 
grid_log_reg.fit(x_train, y_train, groups=groups) 
print('Maximum Cross Validation Accuracy: ',  round(grid_log_reg.best_score_,3))
print('Optimal Parameters: ', grid_log_reg.best_params_)

Maximum Cross Validation Accuracy:  0.322
Optimal Parameters:  {'custom_wt_csp__csp_components': 4, 'logisticregression__C': 0.01}


In [91]:
# for linear svm
param_grid_linear_svm =     {'linearsvc__C' : np.logspace(-4, 3, 16),
                             'custom_wt_csp__csp_components': csp_comps}
grid_linear_svm = GridSearchCV(make_pipeline(Custom_WT_CSP(), StandardScaler(), LinearSVC()), 
                               param_grid=param_grid_linear_svm, cv=cv, scoring=scorer, n_jobs=n_jobs)
grid_linear_svm.fit(x_train, y_train, groups=groups) 
print('Maximum Cross Validation Score: ',  round(grid_linear_svm.best_score_,3))
print('Optimal Parameters: ', grid_linear_svm.best_params_)

Maximum Cross Validation Score:  0.325
Optimal Parameters:  {'custom_wt_csp__csp_components': 4, 'linearsvc__C': 0.0025118864315095794}


In [92]:
# kernel svm
param_grid_kernel_svm = {'svc__C': np.logspace(-4, 3, 8),
                         'svc__gamma': np.logspace(-4, 2, 4) / x_train.shape[0],
                        'custom_wt_csp__csp_components': csp_comps} 
                         
grid_kernel_svm = GridSearchCV(make_pipeline(Custom_WT_CSP(), StandardScaler(), SVC()), 
                            param_grid=param_grid_kernel_svm, cv=cv, scoring=scorer, n_jobs=n_jobs)
grid_kernel_svm.fit(x_train, y_train, groups=groups) 
print('Maximum Cross Validation Accuracy: ',  round(grid_kernel_svm.best_score_,3))
print('Optimal Parameters: ', grid_kernel_svm.best_params_)

Maximum Cross Validation Accuracy:  0.353
Optimal Parameters:  {'custom_wt_csp__csp_components': 4, 'svc__C': 10.0, 'svc__gamma': 0.0015625}


In [93]:
# for lda
shrinkage = list(np.arange(0.0,1.01,0.05))
shrinkage.append('auto')

param_grid_lda = {'lineardiscriminantanalysis__shrinkage': shrinkage,
                  'custom_wt_csp__csp_components': csp_comps}   
grid_lda = GridSearchCV(make_pipeline(Custom_WT_CSP(), StandardScaler(), LDA(solver='eigen')), 
                        param_grid=param_grid_lda, cv=cv, scoring=scorer, n_jobs=n_jobs)
grid_lda.fit(x_train, y_train, groups=groups) 
print('Maximum Cross Validation Score: ',  round(grid_lda.best_score_,3))
print('Optimal Parameters: ', grid_lda.best_params_)

Maximum Cross Validation Score:  0.331
Optimal Parameters:  {'custom_wt_csp__csp_components': 4, 'lineardiscriminantanalysis__shrinkage': 0.2}


## Note
Kernel SVM seems to be the winner. The good thing is that results are stable and distribution is like what we expect but the bad thing is that the kappa values are less than other two methods